# 실험(Experiment) 평가 비교

LangSmith 에서 제공하는 Compare 기능을 활용하여 실험 결과를 쉽게 비교할 수 있습니다.

**참고**
- https://docs.smith.langchain.com/cookbook/tracing-examples/traceable#using-the-decorator

In [ ]:
# 설치
# !pip install -qU langsmith langchain-teddynote

In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH16-Evaluations")

## RAG 성능 테스트를 위한 함수 정의

테스트에 활용할 RAG 시스템을 생성하겠습니다.

In [ ]:
from myrag import PDFRAG
from langchain_openai import ChatOpenAI


# 질문에 대한 답변하는 함수를 생성
def ask_question_with_llm(llm):
    # PDFRAG 객체 생성
    rag = PDFRAG(
        "data/SPRI_AI_Brief_2023년12월호_F.pdf",
        llm,
    )

    # 검색기(retriever) 생성
    retriever = rag.create_retriever()

    # 체인(chain) 생성
    rag_chain = rag.create_chain(retriever)

    def _ask_question(inputs: dict):
        context = retriever.invoke(inputs["question"])
        context = "\n".join([doc.page_content for doc in context])
        return {
            "question": inputs["question"],
            "context": context,
            "answer": rag_chain.invoke(inputs["question"]),
        }

    return _ask_question

In [ ]:
from langchain_ollama import ChatOllama

# Ollama 모델을 불러옵니다.
ollama = ChatOllama(model="EEVE-Korean-10.8B:latest")

# Ollama 모델 호출
ollama.invoke("안녕하세요?")

GPT-4o-mini 모델과 Ollama 모델을 활용하여 질문에 대한 답변을 생성하는 함수를 생성합니다.

In [ ]:
gpt_chain = ask_question_with_llm(ChatOpenAI(model="gpt-4o-mini", temperature=0))
ollama_chain = ask_question_with_llm(ChatOllama(model="EEVE-Korean-10.8B:latest"))

GPT-4o-mini 모델과 Ollama 모델을 활용한 답변 평가를 진행합니다.

2개의 체인에 대하여 각각 진행합니다.

In [ ]:
from langsmith.evaluation import evaluate, LangChainStringEvaluator

# qa 평가자 생성
cot_qa_evalulator = LangChainStringEvaluator(
    "cot_qa",
    config={"llm": ChatOpenAI(model="gpt-4o-mini", temperature=0)},
    prepare_data=lambda run, example: {
        "prediction": run.outputs["answer"],
        "reference": run.outputs["context"],
        "input": example.inputs["question"],
    },
)

dataset_name = "RAG_EVAL_DATASET"

# 평가 실행
experiment_results1 = evaluate(
    gpt_chain,
    data=dataset_name,
    evaluators=[cot_qa_evalulator],
    experiment_prefix="MODEL_COMPARE_EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "GPT-4o-mini 평가 (cot_qa)",
    },
)

# 평가 실행
experiment_results2 = evaluate(
    ollama_chain,
    data=dataset_name,
    evaluators=[cot_qa_evalulator],
    experiment_prefix="MODEL_COMPARE_EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "Ollama(EEVE-Korean-10.8B:latest) 평가 (cot_qa)",
    },
)

결과를 검사하기 위해 비교 보기를 사용하십시오.

**비교 보기를 하는 방법**

![](./assets/eval-01.png)

![](./assets/eval-02.png)

1. Dataset 의 Experiment 탭에서 비교하고 싶은 실험을 선택합니다.
2. 하단의 "Compare" 버튼을 클릭합니다.
3. 비교 보기가 표시됩니다.